## Generate Policies
 - Load source NIST JSON file: Downloads NIST_SP-800-53_rev5_catalog.json from S3 bucket to local directory and loads it into memory
 - Create JSONL file: Processes the JSON catalog to extract controls, skips withdrawn controls, substitutes parameters, and writes structured data to NIST_SP-800-53_rev5_catalog.jsonl
 - Upload JSONL file: Uploads the processed JSONL file back to S3 bucket under the controls/json/ prefix
 - Create markdown files: Reads the JSONL file and generates markdown files organized by control family and in a single directory structure
 - Upload markdown files: Recursively uploads all generated markdown files to S3 bucket under the controls/markdown/ prefix, maintaining directory structure

In [1]:
from datetime import datetime
from IPython.display import Markdown, display
from typing import Any, Dict
import boto3
import json
import os
from pathlib import Path
import shutil
import sys

In [2]:
BUCKET = '183023889407-us-east-1-compliance-rule-generator'
S3_PREFIX_POLICY_MARKDOWN = 'policies/markdown/'

FOLDER_HOME: Path = Path('/home/sagemaker-user')
FOLDER_CONTROL_MARKDOWN: Path = FOLDER_HOME / 'data/controls/markdown/by-family/'
FOLDER_POLICY_MARKDOWN: Path = FOLDER_HOME / 'data/policies/markdown/'

REGION = os.getenv("AWS_REGION", "us-east-1")
s3 = boto3.client("s3", region_name=REGION)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=REGION)

# Bedrock Model Configuration
MODELS = {
    'premium': 'us.anthropic.claude-opus-4-5-20251101-v1:0', # not available
    'good': 'us.anthropic.claude-sonnet-4-5-20250929-v1:0', # times out
    'balanced': 'us.anthropic.claude-sonnet-4-20250514-v1:0',
    'fast_cheap': 'us.anthropic.claude-haiku-4-5-20251001-v1:0',
    'aws_native': 'amazon.nova-premier-v1:0'
}
MODEL_ID = MODELS['balanced']
MAX_TOKENS = 4096
TEMPERATURE = 0.7
PROMPT_TEMPLATE = """
<system>
You are a cybersecurity compliance expert who creates concise, machine-parseable policy documents optimized for RAG systems. Your policies focus on clear rules, explicit validation logic, and scenario patterns that enable LLMs to accurately evaluate compliance scenarios.
</system>

<task>
Generate a concise, RAG-optimized policy document based on the **provided NIST SP 800-53 control**. Focus ONLY on the essential components needed for automated scenario evaluation. The policy will be part of a knowledgebase used by LLMs to determine whether complex scenarios involving multiple policies are compliant or non-compliant.
</task>

<context>
Organization: Large technology company (10,000+ employees) with hybrid cloud infrastructure and regulatory requirements (SOX, FedRAMP, FISMA, PCI-DSS)

RAG Usage: The policy will be chunked and embedded in a vector database. When users present scenarios, relevant policy chunks will be retrieved to determine compliance. The policy must be structured to maximize accurate retrieval and evaluation.
</context>

<output_structure>
Generate the policy using EXACTLY this markdown structure:

markdown
POLICY: [Control ID]: [Control Name]

METADATA
| Field | Value |
|-------|-------|
| Policy ID | POL_[Control ID] |
| NIST Control | [Control ID]: [Control Name] |
| Version | 1.0 |
| Owner | [Appropriate Role] |
| Keywords | keyword1, keyword2, keyword3 |

1. POLICY STATEMENT
Brief (2-3 sentence) summary of what this policy requires.

2. SCOPE
| Entity | In Scope | Notes |
|--------|----------|-------|
| [Entity types] | [YES/NO/CONDITIONAL] | [Brief condition] |

3. KEY ROLES
| Role | Key Responsibilities |
|------|---------------------|
| [Role 1] | [Bullet list of key responsibilities] |
| [Role 2] | [Bullet list of key responsibilities] |

4. RULES
[RULE-01] [Concise rule statement with MUST/MUST NOT/SHALL/SHALL NOT]
[VALIDATION] IF [condition] THEN [outcome]

[RULE-02] [Concise rule statement with MUST/MUST NOT/SHALL/SHALL NOT]
[VALIDATION] IF [condition] THEN [outcome]

[Continue with all essential rules]

5. REQUIRED PROCEDURES
- [PROC-01] [Procedure name] - [Brief description]
- [PROC-02] [Procedure name] - [Brief description]

6. REVIEW REQUIREMENTS
- Policy review frequency: [Timeframe]
- Procedure review frequency: [Timeframe]
- Triggering events: [List of events requiring review]

7. SCENARIO PATTERNS
[SCENARIO-01: Name]
IF [condition1]
AND [condition2]
THEN [compliance outcome]
[Violation severity if applicable]

[SCENARIO-02: Name]
IF [condition1]
AND [condition2]
THEN [compliance outcome]
[Violation severity if applicable]

[Add 3-5 total scenarios that test key policy aspects]

8. COMPLIANCE MAPPING
| Requirement | Rule Reference |
|-------------|---------------|
| [Assessment objective 1] | [RULE-XX] |
| [Assessment objective 2] | [RULE-XX] |
</output_structure>

<formatting_rules>
1. Keep it concise: Each rule should be 1-2 sentences maximum
2. Use precise modal verbs: MUST, MUST NOT, SHALL, SHALL NOT, SHOULD, MAY
3. Include measurable criteria: Specific timeframes, percentages, thresholds
4. Write machine-parseable validation logic: Use IF-THEN format with clear operators
5. Create realistic scenarios: Focus on common compliance questions and edge cases
6. Map to assessment objectives: Ensure every NIST assessment objective links to a rule
7. Use unique identifiers: Each rule needs a unique ID for cross-referencing
</formatting_rules>

<examples>
Example Rule and Validation:

[RULE-03] Access revocation for terminated employees MUST be completed within 24 hours of HR notification for standard terminations and within 1 hour for involuntary terminations.
[VALIDATION] IF employee_status = "terminated" AND termination_type = "standard" AND revocation_time > 24_hours THEN violation
[VALIDATION] IF employee_status = "terminated" AND termination_type = "involuntary" AND revocation_time > 1_hour THEN critical_violation

Example Scenario Pattern:

[SCENARIO-04: Contractor Access After Project]
IF user_type = "contractor"
AND project_end_date < current_date
AND active_access = TRUE
AND exception_documented = FALSE
THEN compliance = FALSE
violation_severity = "Moderate"
</examples>

<input>
Generate a RAG-optimized policy document for the following NIST control:

NIST CONTROL: {{CONTROL_ID}}: {{CONTROL_NAME}}
Family: {{FAMILY}}
Class: SP800-53

Control Statement:
{{CONTROL_STATEMENT}}

Implementation Guidance:
{{IMPLEMENTATION_GUIDANCE}}

Assessment Objectives:
{{ASSESSMENT_OBJECTIVES}}

Related Controls:
{{RELATED_CONTROLS}}
</input>

**provided NIST SP 800-53 control**:
{control_markdown}
"""

In [3]:
def upload_file(local_path: Path, bucket: str, prefix: str):
    """
    Upload a local file to S3.
    """
    key = prefix.rstrip("/") + "/" + local_path.name
    s3.upload_file(str(local_path), bucket, key)
    

In [4]:
def download_file(bucket: str, key: str, local_path: Path):
    """
    Download an S3 object to a local path.
    """
    local_path.parent.mkdir(parents=True, exist_ok=True)
    s3.download_file(bucket, key, str(local_path))
    

In [5]:
def upload_directory_to_s3(source_path: Path, bucket: str, s3_prefix: str):
    """
    Recursively upload all files from source_path to S3 maintaining directory structure.
    """
    source_path = Path(source_path)
    s3_prefix = s3_prefix.rstrip("/")
    
    for file_path in source_path.rglob("*"):
        if file_path.is_file():
            # Calculate relative path from source
            relative_path = file_path.relative_to(source_path)
            # Create S3 key maintaining directory structure
            s3_key = f"{s3_prefix}/{relative_path}".replace("\\", "/")
            
            s3.upload_file(str(file_path), bucket, s3_key)
            print(f"Uploaded: {relative_path} -> s3://{bucket}/{s3_key}")


In [6]:
def print_directory_tree(base_path: Path, max_depth=None):
    def _print_tree(path, prefix="", depth=0):
        if max_depth and depth > max_depth:
            return
        
        items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name))
        
        for i, item in enumerate(items):
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            print(f"{prefix}{current_prefix}{item.name}")
            
            if item.is_dir():
                next_prefix = prefix + ("    " if is_last else "│   ")
                _print_tree(item, next_prefix, depth + 1)
    
    print(base_path.name + "/")
    _print_tree(base_path)


In [7]:
def invoke_bedrock_model(prompt: str, model_id: str = MODEL_ID, max_tokens: int = MAX_TOKENS) -> Dict[str, Any]:
    """Call AWS Bedrock with the given prompt - works with Claude and Nova models"""
    try:
        # Determine model type and prepare appropriate request body
        if 'anthropic' in model_id.lower() or 'claude' in model_id.lower():
            # Claude model format
            request_body = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": max_tokens,
                "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
                "temperature": TEMPERATURE
            }
        elif 'nova' in model_id.lower():
            # Nova models format
            request_body = {
                "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
                "inferenceConfig": {"maxTokens": max_tokens, "temperature": TEMPERATURE}
            }
        else:
            # Generic format fallback - this will need to be tweaked if other models are used
            request_body = {
                "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
                "max_tokens": max_tokens,
                "temperature": TEMPERATURE
            }
        
        # Invoke the model
        response = bedrock_runtime.invoke_model(modelId=model_id, body=json.dumps(request_body))
        response_body = json.loads(response['body'].read())
        
        # Extract text based on model type, currently for anthropic and nova
        if 'nova' in model_id.lower():
            result_text = response_body['output']['message']['content'][0]['text']
        else:
            result_text = response_body['content'][0]['text']
        
        return {
            'success': True,
            'result': result_text,
            'usage': response_body.get('usage', {}),
            'model_id': model_id
        }
    
    except ClientError as e:
        if e.response['Error']['Code'] == 'ThrottlingException':
            print("  ⚠ Rate limit hit, waiting 5 seconds...")
            time.sleep(5)
            return invoke_bedrock_model(prompt, model_id, max_tokens)
        else:
            print(f"  ✗ Bedrock error: {e}")
            return {'success': False, 'error': str(e)}
    
    except Exception as e:
        print(f"  ✗ Unexpected error: {e}")
        return {'success': False, 'error': str(e)}

In [8]:
def create_policy_from_control(control_markdown:str, model_id:str=MODEL_ID) -> str:
    """
    Given markdown of one NIST control, returns a markdown organizational policy based on that control.
    """
    prompt = PROMPT_TEMPLATE.format(control_markdown=control_markdown)
    
    print(f"Calling Bedrock model: {MODEL_ID}")
    result = invoke_bedrock_model(prompt, model_id=MODEL_ID, max_tokens=MAX_TOKENS)
    return result['result']
    

In [9]:
def generate_policies(control_root:Path=FOLDER_CONTROL_MARKDOWN,
    policy_root:Path=FOLDER_POLICY_MARKDOWN, model_id:str=MODEL_ID):
    """
    Traverses NIST controls in local filesystem, generating a corresponding organizational policy
    for each control in a parallel directory structure.
    """   

    # Create two copies of each policy - one set grouped together, another grouped by
    # NIST control category. Each set of generated polices will be subfoldered by date and model id.
    subfolder_name = datetime.now().strftime(f"%Y-%m-%d:%H:%M:%SET_{model_id}")
    all_policies_root: Path = policy_root / "all-controls" / subfolder_name
    all_policies_root.mkdir(parents=True, exist_ok=True)
    policies_by_family_root: Path = policy_root / "by-family" / subfolder_name
    policies_by_family_root.mkdir(parents=True, exist_ok=True)

    for root, dirs, files in os.walk(control_root):
        for control_file_name in files:
            control_absolute_file_path = os.path.join(root, control_file_name)
            control_relative_dir_path = os.path.relpath(root, control_root)
            with open(control_absolute_file_path, 'r') as f:
                control_markdown = f.read()
                policy_markdown = create_policy_from_control(control_markdown=control_markdown,
                    model_id=MODEL_ID)
            for policy_absolute_file_path in [Path(f"{all_policies_root}/{control_relative_dir_path}/policy_{control_file_name}"),
                Path(f"{policies_by_family_root}/{control_relative_dir_path}/policy_{control_file_name}")]:
                
                policy_absolute_file_path.parent.mkdir(parents=True, exist_ok=True)
                policy_absolute_file_path.write_text(policy_markdown)
                print(f"Wrote: {policy_absolute_file_path}")
                # print("-------------------Control-------------------")
                # display(Markdown(control_markdown))
                # print("-------------------Policy-------------------")
                # display(Markdown(policy_markdown))

In [10]:
def main():
    print(f"Running Python: {sys.version.split()[0]}")
    generate_policies(control_root=FOLDER_CONTROL_MARKDOWN, policy_root=FOLDER_POLICY_MARKDOWN,model_id=MODEL_ID)
    # save generated policy markdown files to s3, mirroring directory structure
    upload_directory_to_s3(source_path=FOLDER_POLICY_MARKDOWN, bucket=BUCKET, s3_prefix=S3_PREFIX_POLICY_MARKDOWN)


In [ ]:
main()

Running Python: 3.11.11
Calling Bedrock model: us.anthropic.claude-sonnet-4-20250514-v1:0
Wrote: /home/sagemaker-user/data/policies/markdown/all-controls/2026-01-01:19:25:33ET_us.anthropic.claude-sonnet-4-20250514-v1:0/access-control/policy_AC-1.md
-------------------Control-------------------


# AC-1: Policy and Procedures
            
**Family:** Access Control  
**Class:** SP800-53  
**Keywords:** Assessment, assessment plan, assurance, availability, computer security, confidentiality, control, control assessment, cybersecurity, FISMA, information security, information system, integrity, personally identifiable information, OSCAL, Open Security Controls Assessment Language, Privacy Act, privacy controls, privacy functions, privacy requirements, Risk Management Framework, security controls, security functions, security requirements, system, system security
            
## Control Statement
            
Develop, document, and disseminate to organization-defined personnel or roles: organization-level access control policy that: Addresses purpose, scope, roles, responsibilities, management commitment, coordination among organizational entities, and compliance; and Is consistent with applicable laws, executive orders, directives, regulations, policies, standards, and guidelines; and Procedures to facilitate the implementation of the access control policy and the associated access controls; Designate an an official to manage the access control policy and procedures is defined; to manage the development, documentation, and dissemination of the access control policy and procedures; and Review and update the current access control: Policy the frequency at which the current access control policy is reviewed and updated is defined; and following events that would require the current access control policy reviewed and updated are defined; ; and Procedures the frequency at which the current access control procedures are reviewed and updated is defined; and following events that would require procedures reviewed and updated are defined;.         
            
## Implementation Guidance
            
Access control policy and procedures address the controls in the AC family that are implemented within systems and organizations. The risk management strategy is an important factor in establishing such policies and procedures. Policies and procedures contribute to security and privacy assurance. Therefore, it is important that security and privacy programs collaborate on the development of access control policy and procedures. Security and privacy program policies and procedures at the organization level are preferable, in general, and may obviate the need for mission- or system-specific policies and procedures. The policy can be included as part of the general security and privacy policy or be represented by multiple policies reflecting the complex nature of organizations. Procedures can be established for security and privacy programs, for mission or business processes, and for systems, if needed. Procedures describe how the policies or controls are implemented and can be directed at the individual or role that is the object of the procedure. Procedures can be documented in system security and privacy plans or in one or more separate documents. Events that may precipitate an update to access control policy and procedures include assessment or audit findings, security incidents or breaches, or changes in laws, executive orders, directives, regulations, policies, standards, and guidelines. Simply restating controls does not constitute an organizational policy or procedure.       
            
## Assessment Objectives
            
an access control policy is developed and documented; the access control policy is disseminated to personnel or roles to whom the access control policy is disseminated; access control procedures to facilitate the implementation of the access control policy and associated controls are developed and documented; the access control procedures are disseminated to personnel or roles to whom the access control procedures are disseminated; the organization-level access control policy addresses purpose; the organization-level access control policy addresses scope; the organization-level access control policy addresses roles; the organization-level access control policy addresses responsibilities; the organization-level access control policy addresses management commitment; the organization-level access control policy addresses coordination among organizational entities; the organization-level access control policy addresses compliance; the organization-level access control policy is consistent with applicable laws, Executive Orders, directives, regulations, policies, standards, and guidelines; the an official to manage the access control policy and procedures is defined; is designated to manage the development, documentation, and dissemination of the access control policy and procedures; the current access control policy is reviewed and updated the frequency at which the current access control policy is reviewed and updated is defined;; the current access control policy is reviewed and updated following events that would require the current access control policy reviewed and updated are defined;; the current access control procedures are reviewed and updated the frequency at which the current access control procedures are reviewed and updated is defined;; the current access control procedures are reviewed and updated following events that would require procedures reviewed and updated are defined;.    
            
## Assessment Methods
            
Access control policy and procedures  system security plan  privacy plan  other relevant documents or records Organizational personnel with access control responsibilities  organizational personnel with information security with information security and privacy responsibilities     
            
## Related Controls
            
- ia-1
- pm-9
- pm-24
- ps-8
- si-12


-------------------Policy-------------------


# POLICY: Access Control Policy and Procedures

## Metadata
| Field | Value |
|-------|-------|
| Policy ID | Policy_AC-1 |
| NIST Control | AC-1: Policy and Procedures |
| NIST Family | Access Control (AC) |
| NIST Class | SP 800-53 Rev 5 |
| Owner | Director, Information Security |
| Approver | Chief Information Security Officer |
| Classification | Internal Use Only |

**Related Controls:**
- IA-1, PM-9, PM-24, PS-8, SI-12

**Keywords:** `access control`, `policy development`, `procedures`, `governance`, `compliance`, `FISMA`, `FedRAMP`, `SOX`, `PCI-DSS`, `GDPR`, `security controls`, `privacy controls`, `risk management`, `organizational policy`, `security program`, `privacy program`, `management commitment`, `coordination`, `documentation`, `dissemination`, `review`, `update`, `assessment`, `audit`

---

## 1. PURPOSE

### 1.1 Policy Intent
[RULE] This policy establishes comprehensive requirements for developing, implementing, maintaining, and governing access control policies and procedures across the organization to ensure consistent protection of information systems and data assets in compliance with applicable laws, regulations, and organizational standards.

### 1.2 Business Objectives
[OBJECTIVE-1] Establish standardized access control governance framework across all business units and subsidiaries
[OBJECTIVE-2] Ensure compliance with regulatory requirements including FISMA, FedRAMP, SOX, PCI-DSS, and GDPR
[OBJECTIVE-3] Define clear roles, responsibilities, and accountability for access control program management
[OBJECTIVE-4] Implement systematic review and update processes to maintain policy currency and effectiveness
[OBJECTIVE-5] Facilitate consistent implementation of access controls across hybrid cloud and on-premises environments
[OBJECTIVE-6] Enable automated compliance validation and continuous monitoring capabilities

---

## 2. SCOPE

### 2.1 Applicability Matrix
| Entity Type | In Scope | Conditions |
|-------------|----------|------------|
| All Employees | Yes | Including full-time, part-time, contractors, and temporary staff |
| All Business Units | Yes | Including subsidiaries and joint ventures where organization has controlling interest |
| All Information Systems | Yes | Including cloud services, on-premises systems, and hybrid environments |
| Third-Party Service Providers | Yes | When processing organizational data or accessing organizational systems |
| Board Members | Yes | For systems access and data handling requirements |
| Customers/Partners | Partial | Only for systems they are granted access to |

### 2.2 System Scope
| System Category | In Scope | Notes |
|----------------|----------|-------|
| SOX-Scoped Systems | Yes | Enhanced controls and audit requirements |
| FedRAMP Systems | Yes | Must meet FedRAMP Moderate/High baselines |
| PCI-DSS Systems | Yes | Cardholder data environment protections |
| GDPR Systems | Yes | Personal data processing systems |
| Development Systems | Yes | Including CI/CD pipelines and repositories |
| Corporate IT Systems | Yes | Email, collaboration, HR systems |
| OT/IoT Systems | Yes | Operational technology and IoT devices |

### 2.3 Exclusions
[EXCLUSION-1] Publicly available information systems with no authentication requirements
[EXCLUSION-2] Systems scheduled for decommissioning within 90 days (with CISO approval)
[EXCLUSION-3] Emergency response systems during declared emergencies (temporary, requires post-event review)

---

## 3. ROLES AND RESPONSIBILITIES

### 3.1 Designated Official
[RULE-AC-001] The Chief Information Security Officer (CISO) SHALL serve as the designated official responsible for managing access control policy and procedures development, documentation, and dissemination.

| Role | Designated Official |
|------|-------------------|
| Title | Chief Information Security Officer (CISO) |
| Responsibility | Overall access control program governance |
| Authority | Policy approval, exception authorization, resource allocation |
| Accountability | Board and executive reporting on access control program effectiveness |
| Delegation | May delegate specific implementation responsibilities but retains ultimate accountability |

### 3.2 Access Control Policy Manager
| Responsibility | Measurable Criteria |
|----------------|-------------------|
| Policy Development | [MEASURABLE] Complete policy reviews within 30 days of triggering events |
| Documentation Management | [MEASURABLE] Maintain 99.5% policy document availability in central repository |
| Dissemination Coordination | [MEASURABLE] Achieve 100% acknowledgment within required timeframes |
| Stakeholder Engagement | [MEASURABLE] Conduct quarterly stakeholder review meetings |

### 3.3 Business Unit Security Officers
[RESPONSIBILITY] Implement organization-level access control policies within their business units
[RESPONSIBILITY] Coordinate with central security team on policy interpretation and exceptions
[RESPONSIBILITY] Report policy violations and improvement recommendations
[MEASURABLE] Monthly compliance reporting with <5% variance from targets
[MEASURABLE] Exception request processing within 5 business days

---

## 4. POLICY STATEMENTS

### 4.1 Access Control Policy Development
#### 4.1.1 Policy Framework Requirements
[RULE-AC-002] The organization MUST develop and maintain a comprehensive organization-level access control policy that addresses all required elements as specified in this section.
[VALIDATION] IF access_control_policy NOT_EXISTS OR required_elements_coverage < 100% THEN violation

[RULE-AC-003] The access control policy MUST address the following elements: purpose, scope, roles, responsibilities, management commitment, coordination among organizational entities, and compliance requirements.
[VALIDATION] IF policy_elements NOT_IN ["purpose", "scope", "roles", "responsibilities", "management_commitment", "coordination", "compliance"] THEN violation

#### 4.1.2 Regulatory Consistency Requirements
[RULE-AC-004] The access control policy MUST be consistent with applicable laws, executive orders, directives, regulations, policies, standards, and guidelines including but not limited to FISMA, FedRAMP, SOX, PCI-DSS, and GDPR.
[VALIDATION] IF regulatory_alignment_review_date > 12_months_ago THEN violation

[RULE-AC-005] Policy updates MUST be reviewed by Legal and Compliance teams within 10 business days of draft completion to ensure regulatory consistency.
[VALIDATION] IF legal_review_days > 10 AND compliance_review_days > 10 THEN violation

### 4.2 Procedure Development and Documentation
#### 4.2.1 Implementation Procedures
[RULE-AC-006] The organization MUST develop, document, and maintain detailed procedures to facilitate implementation of the access control policy and associated access controls.
[VALIDATION] IF procedure_coverage < 95% OF access_controls THEN violation

[RULE-AC-007] Access control procedures MUST include step-by-step implementation guidance, role assignments, and success criteria for each covered control.
[VALIDATION] IF procedure_completeness_score < 90% THEN violation

#### 4.2.2 Procedure Integration
[RULE-AC-008] Procedures MUST be integrated with existing business processes and technical implementation guides to ensure practical applicability.
[VALIDATION] IF business_process_integration = FALSE OR technical_integration = FALSE THEN violation

### 4.3 Dissemination Requirements
#### 4.3.1 Distribution Matrix
[RULE-AC-009] Access control policies and procedures MUST be disseminated to all personnel and roles with access control responsibilities within 15 business days of approval or update.
[VALIDATION] IF dissemination_days > 15 THEN violation

[RULE-AC-010] Dissemination MUST include acknowledgment tracking with 100% acknowledgment required within 30 days of distribution.
[VALIDATION] IF acknowledgment_rate < 100% AND days_since_distribution > 30 THEN violation

#### 4.3.2 Accessibility Requirements
[RULE-AC-011] Policies and procedures MUST be accessible through the organization's central policy repository with 99.9% uptime availability.
[VALIDATION] IF repository_uptime < 99.9% THEN violation

### 4.4 Management and Governance
#### 4.4.1 Designated Official Management
[RULE-AC-012] The designated official (CISO) MUST maintain oversight of access control policy and procedure development, documentation, and dissemination activities.
[VALIDATION] IF ciso_oversight_review_frequency != "monthly" THEN violation

[RULE-AC-013] The designated official MUST provide quarterly reports to executive leadership on access control program effectiveness and policy compliance.
[VALIDATION] IF quarterly_report_days_late > 5 THEN violation

---

## 5. PROCEDURES

### 5.1 Procedure Documentation Requirements
[RULE-PROC-001] All access control procedures MUST follow the organization's standard operating procedure template and include version control, approval signatures, and effective dates.
[VALIDATION] IF procedure_template_compliance < 100% THEN violation

[RULE-PROC-002] Procedures MUST include automated validation steps where technically feasible, with manual verification requirements clearly specified.
[VALIDATION] IF automation_opportunity_score < 70% AND manual_validation_undefined = TRUE THEN violation

### 5.2 Required Procedures
| Procedure ID | Title | Owner | Review Frequency |
|--------------|-------|-------|------------------|
| PROC-AC-001 | Access Control Policy Development and Review | Security Policy Manager | Annually |
| PROC-AC-002 | Policy Dissemination and Acknowledgment | Communications Team | Annually |
| PROC-AC-003 | Access Control Exception Management | CISO Office | Semi-annually |
| PROC-AC-004 | Regulatory Compliance Validation | Legal and Compliance | Annually |
| PROC-AC-005 | Stakeholder Coordination and Communication | Security Governance | Annually |
| PROC-AC-006 | Policy Training and Awareness | Security Training Team | Annually |

---

## 6. REVIEW AND UPDATE REQUIREMENTS

### 6.1 Scheduled Reviews
[RULE-REV-001] Access control policies MUST be reviewed and updated annually or more frequently as determined by the CISO.
[VALIDATION] IF policy_review_date > 365_days_ago THEN violation

[RULE-REV-002] Access control procedures MUST be reviewed and updated annually or when associated policies are updated.
[VALIDATION] IF procedure_review_date > 365_days_ago OR procedure_update_lag > 30_days THEN violation

| Review Type | Frequency | Owner | Deliverable |
|-------------|-----------|-------|-------------|
| Comprehensive Policy Review | Annually | Security Policy Manager | Updated policy document with change log |
| Regulatory Alignment Review | Semi-annually | Legal and Compliance | Compliance gap analysis and remediation plan |
| Effectiveness Assessment | Quarterly | Security Governance | Metrics dashboard and improvement recommendations |
| Stakeholder Feedback Review | Quarterly | Business Unit Security Officers | Consolidated feedback report and response plan |

### 6.2 Event-Driven Reviews
[RULE-REV-003] Access control policies and procedures MUST be reviewed and updated within 60 days following significant organizational changes, security incidents, audit findings, or regulatory updates.
[VALIDATION] IF triggering_event_date + 60_days < current_date AND review_not_initiated = TRUE THEN violation

| Triggering Event | Review Scope | Timeline | Owner |
|------------------|--------------|----------|-------|
| Major Security Incident | Full policy and affected procedures | 30 days | CISO |
| Audit Findings (High/Critical) | Specific areas identified | 45 days | Security Policy Manager |
| Regulatory Changes | Affected policy sections | 60 days | Legal and Compliance |
| Organizational Restructuring | Roles and responsibilities sections | 30 days | CISO |
| Technology Platform Changes | Implementation procedures | 45 days | Technical Security Team |

[VALIDATION] IF event_driven_review_completion > specified_timeline THEN violation_severity = "Serious"

---

## 7. DISSEMINATION REQUIREMENTS

### 7.1 Distribution Matrix
[RULE-DIS-001] Policy dissemination MUST use multiple communication channels to ensure comprehensive organizational coverage.
[VALIDATION] IF communication_channels < 3 THEN violation

| Audience | Method | Acknowledgment | Timeline |
|----------|--------|----------------|----------|
| All Employees | Email + Intranet Portal | Required | 15 business days |
| Security Personnel | Direct notification + Training session | Required + Assessment | 10 business days |
| Management (Director+) | Executive briefing + Document review | Required | 5 business days |
| Board Members | Board package + Presentation | Noted in minutes | Next scheduled meeting |
| Third-party providers | Contract addendum + Notification | Signed acknowledgment | 20 business days |

### 7.2 Training Requirements
[RULE-DIS-002] All personnel with access control responsibilities MUST complete policy training within 30 days of policy updates.
[VALIDATION] IF training_completion_days > 30 THEN violation

| Role | Training Module | Duration | Passing Score | Frequency |
|------|-----------------|----------|---------------|-----------|
| All Employees | Access Control Awareness | 30 minutes | 80% | Annually |
| Security Personnel | Access Control Implementation | 2 hours | 90% | Annually |
| Managers | Access Control Management | 1 hour | 85% | Annually |
| Auditors | Access Control Assessment | 3 hours | 95% | Annually |

[VALIDATION] IF training_completion_rate < 95% AND days_since_policy_update > 45 THEN violation

---

## 8. COMPLIANCE VALIDATION

### 8.1 Key Performance Indicators
| KPI | Target | Measurement | Frequency |
|-----|--------|-------------|-----------|
| Policy Acknowledgment Rate | 100% | Percentage of required personnel acknowledging within timeline | Monthly |
| Policy Review Timeliness | 100% | Percentage of reviews completed within scheduled timeframes | Quarterly |
| Exception Processing Time | ≤5 business days | Average days from request to decision | Monthly |
| Training Completion Rate | ≥95% | Percentage of required personnel completing training | Quarterly |
| Audit Finding Closure | ≤60 days | Average days to close policy-related findings | Quarterly |
| Stakeholder Satisfaction | ≥4.0/5.0 | Survey score on policy clarity and usefulness | Semi-annually |

### 8.2 Validation Methods
#### 8.2.1 Continuous Monitoring
| Control | Tool | Alert Threshold | Response SLA |
|---------|------|-----------------|--------------|
| Policy Acknowledgment | HR System Integration | <90% within 20 days | 4 hours |
| Document Availability | Repository Monitoring | >1 hour downtime | 15 minutes |
| Training Completion | LMS Integration | <80% within 25 days | 24 hours |
| Review Schedule Adherence | Workflow System | 7 days before due date | 2 business days |

#### 8.2.2 Periodic Assessments
| Assessment | Scope | Frequency | Assessor |
|------------|-------|-----------|----------|
| Policy Effectiveness Review | Full program assessment | Annually | Internal Audit |
| Compliance Gap Analysis | Regulatory alignment | Semi-annually | Legal and Compliance |
| Implementation Maturity Assessment | Procedure effectiveness | Annually | Security Governance |
| Stakeholder Feedback Assessment | User experience and clarity | Quarterly | Business Units |

---

## 9. EXCEPTIONS

### 9.1 Exception Process
[RULE-EXC-001] All exceptions to access control policy requirements MUST be formally documented, justified, and approved through the established exception process.
[VALIDATION] IF exception_exists = TRUE AND formal_documentation = FALSE THEN violation

[RULE-EXC-002] Exception requests MUST include business justification, risk assessment, compensating controls, and proposed duration.
[VALIDATION] IF exception_request_completeness < 100% THEN request_rejected = TRUE

[RULE-EXC-003] All exceptions MUST be reviewed monthly and renewed annually with updated risk assessments.
[VALIDATION] IF exception_review_date > 30_days_ago OR exception_age > 365_days THEN violation

### 9.2 Approval Authority
| Risk Level | Approver | Max Duration | Conditions |
|------------|----------|--------------|------------|
| Low | Security Manager | 12 months | Compensating controls required |
| Medium | CISO | 6 months | Executive sponsor required |
| High | CISO + CRO | 3 months | Board notification required |
| Critical | Executive Committee | 30 days | Emergency use only |

[VALIDATION] IF risk_level = "High" AND board_notification = FALSE THEN violation
[VALIDATION] IF risk_level = "Critical" AND duration > 30_days THEN violation

---

## 10. ENFORCEMENT

### 10.1 Violation Classification
| Severity | Examples |

Wrote: /home/sagemaker-user/data/policies/markdown/by-family/2026-01-01:19:25:33ET_us.anthropic.claude-sonnet-4-20250514-v1:0/access-control/policy_AC-1.md
-------------------Control-------------------


...

Wrote: /home/sagemaker-user/data/policies/markdown/by-family/2026-01-01:19:25:33ET_us.anthropic.claude-sonnet-4-20250514-v1:0/access-control/policy_AC-2.2.md
-------------------Control-------------------


# AC-2.2: Automated Temporary and Emergency Account Management
            
**Family:** Access Control  
**Class:** SP800-53-enhancement  
**Keywords:** Assessment, assessment plan, assurance, availability, computer security, confidentiality, control, control assessment, cybersecurity, FISMA, information security, information system, integrity, personally identifiable information, OSCAL, Open Security Controls Assessment Language, Privacy Act, privacy controls, privacy functions, privacy requirements, Risk Management Framework, security controls, security functions, security requirements, system, system security
            
## Control Statement
            
Automatically remove temporary and emergency accounts after the time period after which to automatically remove or disable temporary or emergency accounts is defined;.         
            
## Implementation Guidance
            
Management of temporary and emergency accounts includes the removal or disabling of such accounts automatically after a predefined time period rather than at the convenience of the system administrator. Automatic removal or disabling of accounts provides a more consistent implementation.       
            
## Assessment Objectives
            
temporary and emergency accounts are automatically remove after the time period after which to automatically remove or disable temporary or emergency accounts is defined;.    
            
## Assessment Methods
            
Access control policy  procedures for addressing account management  system design documentation  system configuration settings and associated documentation  system-generated list of temporary accounts removed and/or disabled  system-generated list of emergency accounts removed and/or disabled  system audit records  system security plan  other relevant documents or records Organizational personnel with account management responsibilities  system/network administrators  organizational personnel with information security with information security responsibilities  system developers Automated mechanisms for implementing account management functions     
            
## Related Controls
            



-------------------Policy-------------------


# POLICY: Automated Temporary and Emergency Account Management

## Metadata
| Field | Value |
|-------|-------|
| Policy ID | Policy_AC-2.2 |
| NIST Control | AC-2.2: Automated Temporary and Emergency Account Management |
| NIST Family | Access Control (AC) |
| NIST Class | SP 800-53 Rev 5 |
| Owner | Director of Identity and Access Management |
| Approver | Chief Information Security Officer |
| Classification | Internal Use Only |

**Related Controls:**
AC-2, AC-2.1, AC-2.3, AC-2.4, AC-2.5, AC-2.6, AC-2.7, AC-2.8, AC-2.9, AC-2.10, AC-2.11, AC-2.12, AC-2.13

**Keywords:** `automated account management`, `temporary accounts`, `emergency accounts`, `account lifecycle`, `access control`, `privileged access`, `identity management`, `account expiration`, `automated removal`, `security automation`, `account governance`, `time-based access`, `incident response`, `compliance automation`, `identity lifecycle`

---

## 1. PURPOSE

### 1.1 Policy Intent
[RULE] This policy establishes requirements for automated management of temporary and emergency accounts to ensure consistent, timely removal or disabling without reliance on manual administrator intervention.

### 1.2 Business Objectives
[OBJECTIVE-1] Minimize security risk exposure from orphaned temporary and emergency accounts
[OBJECTIVE-2] Ensure consistent application of account lifecycle management across all systems
[OBJECTIVE-3] Reduce administrative overhead through automation of account management processes
[OBJECTIVE-4] Maintain compliance with regulatory requirements for access control
[OBJECTIVE-5] Enable rapid response capabilities while maintaining security controls
[OBJECTIVE-6] Provide comprehensive audit trails for temporary and emergency access

---

## 2. SCOPE

### 2.1 Applicability Matrix
| Entity Type | In Scope | Conditions |
|-------------|----------|------------|
| Information Systems | Yes | All systems processing organizational data |
| Cloud Services | Yes | All SaaS, PaaS, IaaS platforms |
| Network Infrastructure | Yes | Routers, switches, firewalls, load balancers |
| Database Systems | Yes | All RDBMS, NoSQL, data warehouses |
| Applications | Yes | Custom and COTS applications |
| Identity Providers | Yes | Active Directory, LDAP, cloud identity services |
| Privileged Access Management | Yes | All PAM solutions and jump servers |

### 2.2 System Scope
| System Category | In Scope | Notes |
|-----------------|----------|-------|
| High Impact Systems | Yes | Maximum 4-hour account lifetime |
| Moderate Impact Systems | Yes | Maximum 24-hour account lifetime |
| Low Impact Systems | Yes | Maximum 72-hour account lifetime |
| Development Systems | Yes | Maximum 7-day account lifetime |
| Test Systems | Yes | Maximum 30-day account lifetime |
| Sandbox Systems | Partial | Subject to data classification requirements |

### 2.3 Exclusions
[EXCLUSION-1] Break-glass accounts with manual oversight approval processes
[EXCLUSION-2] Service accounts with indefinite operational requirements (subject to separate policy)
[EXCLUSION-3] Legacy systems with documented technical limitations (requires compensating controls)
[EXCLUSION-4] Systems in decommissioning status (within 90 days of retirement)

---

## 3. ROLES AND RESPONSIBILITIES

### 3.1 Designated Official
[RULE] Each business unit MUST designate an Account Management Official responsible for automated account lifecycle implementation and oversight.

| Role | Responsibilities | Measurable Criteria | Authority Level |
|------|------------------|-------------------|-----------------|
| Account Management Official | Policy implementation oversight | 99.5% automation compliance rate | Business unit scope |
| Identity and Access Management Team | Technical implementation and monitoring | <15 minute response to automation failures | Enterprise scope |
| System Administrators | System-specific configuration | 100% system coverage within 30 days | System scope |
| Security Operations Center | 24/7 monitoring and alerting | Alert response within 5 minutes | Enterprise scope |
| Incident Response Team | Emergency account authorization | Decision within 30 minutes of request | Enterprise scope |

---

## 4. POLICY STATEMENTS

### 4.1 Automated Account Lifecycle Management

#### 4.1.1 Temporary Account Automation
[RULE-AC-001] All temporary accounts MUST be automatically removed or disabled upon expiration without manual intervention.
[VALIDATION] IF account_type = "temporary" AND current_time >= expiration_time AND account_status = "active" THEN violation_severity = "High"

[RULE-AC-002] Temporary account maximum lifetimes MUST be enforced based on system impact level:
- High Impact: 4 hours maximum
- Moderate Impact: 24 hours maximum  
- Low Impact: 72 hours maximum
[VALIDATION] IF account_type = "temporary" AND (expiration_time - creation_time) > max_lifetime_by_impact THEN violation_severity = "Serious"

[RULE-AC-003] Temporary account creation MUST include automated expiration timestamp and business justification.
[VALIDATION] IF account_type = "temporary" AND (expiration_time NOT_EXISTS OR justification NOT_EXISTS) THEN violation_severity = "Serious"

#### 4.1.2 Emergency Account Automation
[RULE-AC-004] Emergency accounts MUST be automatically disabled within 24 hours of creation unless explicitly extended through approval workflow.
[VALIDATION] IF account_type = "emergency" AND (current_time - creation_time) > 24_hours AND account_status = "active" AND extension_approved != TRUE THEN violation_severity = "Critical"

[RULE-AC-005] Emergency account extensions MUST NOT exceed 72 hours total lifetime and require CISO or Deputy CISO approval.
[VALIDATION] IF account_type = "emergency" AND total_lifetime > 72_hours THEN violation_severity = "Critical"
[VALIDATION] IF account_type = "emergency" AND extension_requested = TRUE AND approver_role NOT_IN ["CISO", "Deputy_CISO"] THEN violation_severity = "Serious"

[RULE-AC-006] Emergency accounts MUST be tied to active incident tickets and automatically disabled upon incident closure.
[VALIDATION] IF account_type = "emergency" AND incident_status = "closed" AND account_status = "active" THEN violation_severity = "High"

#### 4.1.3 Automation Implementation Requirements
[RULE-AC-007] Account lifecycle automation MUST be implemented using centralized identity management systems with redundant processing capabilities.
[VALIDATION] IF automation_system_availability < 99.9% THEN violation_severity = "Serious"

[RULE-AC-008] Automation failures MUST trigger immediate alerts to Security Operations Center with 5-minute response SLA.
[VALIDATION] IF automation_failure_detected = TRUE AND alert_sent = FALSE THEN violation_severity = "Critical"
[VALIDATION] IF automation_failure_detected = TRUE AND response_time > 5_minutes THEN violation_severity = "High"

[RULE-AC-009] All automated account actions MUST generate audit logs with timestamp, account identifier, action taken, and system identifier.
[VALIDATION] IF automated_action_occurred = TRUE AND audit_log NOT_EXISTS THEN violation_severity = "Serious"

#### 4.1.4 Monitoring and Alerting
[RULE-AC-010] Real-time monitoring MUST detect and alert on accounts exceeding defined lifetimes within 15 minutes.
[VALIDATION] IF account_lifetime_exceeded = TRUE AND alert_time > (detection_time + 15_minutes) THEN violation_severity = "High"

[RULE-AC-011] Daily reports MUST be generated showing all temporary and emergency account creation, modification, and removal activities.
[VALIDATION] IF report_generation_date != current_date THEN violation_severity = "Moderate"

---

## 5. PROCEDURES

### 5.1 Procedure Documentation Requirements
[RULE-PROC-001] Each system MUST maintain documented procedures for automated account lifecycle implementation.
[RULE-PROC-002] Procedures MUST be reviewed quarterly and updated within 30 days of system changes.
[RULE-PROC-003] All procedures MUST include rollback and manual override processes for automation failures.

### 5.2 Required Procedures
| Procedure ID | Title | Owner | Review Frequency |
|--------------|-------|-------|------------------|
| PROC-AC-001 | Temporary Account Automation Configuration | IAM Team | Quarterly |
| PROC-AC-002 | Emergency Account Lifecycle Management | IAM Team | Quarterly |
| PROC-AC-003 | Automation Failure Response | SOC | Quarterly |
| PROC-AC-004 | Account Extension Approval Workflow | Security Team | Semi-annually |
| PROC-AC-005 | Audit Log Analysis for Account Management | Compliance Team | Annually |

---

## 6. REVIEW AND UPDATE REQUIREMENTS

### 6.1 Scheduled Reviews
[RULE-REV-001] This policy MUST be reviewed annually and updated within 60 days of review completion.
[RULE-REV-002] Technical implementation standards MUST be reviewed quarterly.

| Review Type | Frequency | Owner | Deliverable |
|-------------|-----------|-------|-------------|
| Policy Review | Annual | CISO | Updated policy document |
| Technical Standards Review | Quarterly | Director of IAM | Technical standards update |
| Compliance Assessment | Semi-annual | Compliance Officer | Assessment report |
| Effectiveness Review | Quarterly | Security Architecture | Metrics dashboard |

### 6.2 Event-Driven Reviews
[RULE-REV-003] Policy MUST be reviewed within 30 days of security incidents involving temporary or emergency accounts.
[RULE-REV-004] Policy MUST be reviewed within 60 days of regulatory changes affecting access control requirements.

| Triggering Event | Review Scope | Timeline | Owner |
|------------------|--------------|----------|-------|
| Security Incident | Full policy review | 30 days | CISO |
| Regulatory Change | Compliance alignment | 60 days | Compliance Officer |
| System Architecture Change | Technical requirements | 45 days | Director of IAM |
| Audit Findings | Affected sections | 30 days | Policy Owner |

[VALIDATION] IF triggering_event_occurred = TRUE AND review_completed = FALSE AND days_elapsed > timeline THEN violation_severity = "Moderate"

---

## 7. DISSEMINATION REQUIREMENTS

### 7.1 Distribution Matrix
[RULE-DIS-001] Policy updates MUST be distributed to all stakeholders within 15 business days of approval.
[RULE-DIS-002] All personnel with account management responsibilities MUST acknowledge receipt within 10 business days.

| Audience | Method | Acknowledgment | Timeline |
|----------|--------|----------------|----------|
| System Administrators | Email + Portal | Required | 10 business days |
| Security Team | Direct briefing | Required | 5 business days |
| Compliance Team | Document repository | Required | 10 business days |
| Business Unit Leaders | Executive summary | Required | 15 business days |
| All Employees | Company portal | Not required | 30 business days |

### 7.2 Training Requirements
| Role | Training Module | Duration | Passing Score | Frequency |
|------|-----------------|----------|---------------|-----------|
| System Administrators | Account Lifecycle Automation | 2 hours | 85% | Annual |
| Security Personnel | Emergency Access Management | 1 hour | 90% | Annual |
| Incident Response Team | Emergency Account Procedures | 1 hour | 90% | Semi-annual |
| Managers | Access Control Oversight | 30 minutes | 80% | Annual |

[VALIDATION] IF role_requires_training = TRUE AND training_completed = FALSE AND hire_date > 30_days THEN violation_severity = "Moderate"

---

## 8. COMPLIANCE VALIDATION

### 8.1 Key Performance Indicators
| KPI | Target | Measurement | Frequency |
|-----|--------|-------------|-----------|
| Automation Success Rate | ≥99.5% | (Successful automated actions / Total required actions) × 100 | Daily |
| Account Lifetime Compliance | 100% | Accounts removed within defined timeframes | Daily |
| Alert Response Time | ≤5 minutes | Time from automation failure to SOC response | Continuous |
| Manual Override Rate | ≤0.5% | Manual interventions / Total account actions | Weekly |
| Audit Trail Completeness | 100% | Complete logs / Total automated actions | Daily |

### 8.2 Validation Methods

#### 8.2.1 Continuous Monitoring
| Control | Tool | Alert Threshold | Response SLA |
|---------|------|----------------|--------------|
| Account Expiration | SIEM + IAM System | Account active >15 min past expiration | 15 minutes |
| Automation Failure | Monitoring Platform | Any automation process failure | 5 minutes |
| Unauthorized Extensions | IAM System | Extension without proper approval | 30 minutes |
| Missing Audit Logs | Log Management | Gap in audit trail >5 minutes | 1 hour |

#### 8.2.2 Periodic Assessments
| Assessment | Scope | Frequency | Assessor |
|------------|-------|-----------|----------|
| Technical Controls Review | All systems | Monthly | Internal Security Team |
| Process Compliance Audit | Sample of accounts | Quarterly | Internal Audit |
| Automation Effectiveness | Full environment | Semi-annually | External Assessor |
| Regulatory Compliance | SOX/FedRAMP systems | Annually | External Auditor |

---

## 9. EXCEPTIONS

### 9.1 Exception Process
[RULE-EXC-001] Exception requests MUST include business justification, risk assessment, and compensating controls.
[RULE-EXC-002] All exceptions MUST be reviewed and re-approved annually.
[RULE-EXC-003] Exception approvals MUST be documented with specific conditions and monitoring requirements.

### 9.2 Approval Authority
| Risk Level | Approver | Max Duration |
|------------|----------|--------------|
| Low | Director of IAM | 12 months |
| Moderate | CISO | 6 months |
| High | CISO + CTO | 3 months |
| Critical | Executive Committee | 30 days |

[VALIDATION] IF exception_risk_level = "High" AND approver NOT_IN ["CISO", "CTO"] THEN violation_severity = "Serious"
[VALIDATION] IF exception_duration > max_duration_by_risk THEN violation_severity = "Moderate"

---

## 10. ENFORCEMENT

### 10.1 Violation Classification
| Severity | Examples | Consequences |
|----------|----------|--------------|
| Critical | Emergency account active >72 hours without approval | Immediate escalation to CISO, mandatory incident response |
| Serious | Temporary account exceeding maximum lifetime | Security team investigation, process review |
| High | Automation failure not addressed within SLA | SOC escalation, technical review |
| Moderate | Missing documentation or delayed training | Management notification, corrective action plan |

### 10.2 Escalation Path
[ESCALATION] Critical violations: Immediate notification to CISO and incident response activation
[ESCALATION] Serious violations: Notification to Director of IAM within 2 hours, investigation within 24 hours
[ESCALATION] High violations: Notification to Security Operations Manager within 4 hours
[ESCALATION] Moderate violations: Notification to appropriate manager within 24 hours

---

## 11. SCENARIO VALIDATION RULES

### 11.1 Common Scenario Patterns

[SCENARIO: Emergency Account During Security Incident]
IF incident_declared = TRUE
AND account_type = "emergency"
AND incident_commander_approval = TRUE
AND business_justification EXISTS
THEN emergency_account_creation = ALLOWED
AND max_lifetime = 24_hours
AND extension_requires_ciso_approval = TRUE
EXPECTED_SLA = 30_minutes_for_approval
IF account_active_time > 24_hours AND extension_not_approved THEN violation_severity = "Critical"

[SCENARIO: Temporary Account for Contractor Access]
IF user_type = "contractor"
AND account_type = "temporary"
AND contract_end_date EXISTS
AND manager_approval = TRUE
THEN account_creation = ALLOWED
AND expiration_date = MIN(contract_end_date, system_max_lifetime)
AND automated_removal = REQUIRED
EXPECTED_SLA = 15_minutes_past_expiration
IF current_time > (expiration_date + 15_minutes) AND account_status = "active" THEN violation_severity = "High"

[SCENARIO: Automation System Failure During Off-Hours]
IF automation_system_status = "failed"
AND current_time BETWEEN 18:00 AND 

Calling Bedrock model: us.anthropic.claude-sonnet-4-20250514-v1:0
